In [ ]:
# Fetching the tables from the Azure Sql Server using JDBC driver
driver = "com.microsoft.sqlserver.jdbc.SQLServerDriver"
url = "jdbc:sqlserver://<your_server>.database.windows.net:1433;database=<your_database>"
table = "dbo.train"
table1= "dbo.store"
table2= "dbo.test"
user = "<user>@<server>"
password = "<password>"
train = spark.read.format("jdbc").option("driver", driver).option("url", url).option("dbtable", table).option("user", user).option("password", password).load()
store = spark.read.format("jdbc").option("driver", driver).option("url", url).option("dbtable", table1).option("user", user).option("password", password).load()
#test1 = spark.read.format("jdbc").option("driver", driver).option("url", url).option("dbtable", table2).option("user", user).option("password", password).load()

In [ ]:
# libraries required
import numpy as np
import pandas as pd
from pyspark.sql.types import IntegerType,FloatType
from pyspark.ml.feature import StringIndexer,VectorAssembler
from pyspark.ml.regression import LinearRegression, LinearRegressionSummary
from pyspark.mllib.regression import LabeledPoint, LinearRegressionWithSGD
from pyspark.mllib.evaluation import RegressionMetrics
import os

In [ ]:
store.registerTempTable('store')
query = sqlContext.sql('SELECT percentile_approx(CompetitionDistance, 0.5) as out FROM store')
val1 = query.collect()[0][0]
test1 = test1.na.fill({'Open': 0})
store = store.na.fill({'CompetitionDistance': val1,
                       'CompetitionOpenSinceMonth': 0,
                       'CompetitionOpenSinceYear':0,
                       'Promo2SinceWeek':0,
                       'Promo2SinceYear':0,
                       'PromoInterval':0})

In [ ]:
mereged_train_df = train.join(store, on=['Store'], how='inner')
mereged_test_df = test1.join(store, on=['Store'], how='inner')

In [ ]:
mereged_train_df.registerTempTable('train')
mereged_train_df = sqlContext.sql('select *,day(Date) as Day, month(Date) as Month, year(Date) as Year from train')
mereged_test_df.registerTempTable('test1')
mereged_test_df = sqlContext.sql('select *,day(Date) as Day, month(Date) as Month, year(Date) as Year from test1')

mereged_train_df1= mereged_train_df.toPandas()
mereged_test_df1=mereged_test_df.toPandas()

In [ ]:
# As we are using LinearRegression
mappings = {'0':0, 'a':1, 'b':2, 'c':3, 'd':4}
mereged_train_df1["StoreType"] = mereged_train_df1["StoreType"].map(mappings)
mereged_train_df1["Assortment"] = mereged_train_df1["Assortment"].map(mappings)
mereged_train_df1["StateHoliday"].loc[mereged_train_df1["StateHoliday"] == 0] = "0"
mereged_train_df1["StateHoliday"] = mereged_train_df1["StateHoliday"].map(mappings)
mereged_test_df1["StoreType"] = mereged_test_df1["StoreType"].map(mappings)
mereged_test_df1["Assortment"] = mereged_test_df1["Assortment"].map(mappings)
mereged_test_df1["StateHoliday"].loc[mereged_test_df1["StateHoliday"] == 0] = "0"
mereged_test_df1["StateHoliday"] = mereged_test_df1["StateHoliday"].map(mappings)

mereged_train_df = spark.createDataFrame(mereged_train_df1)
mereged_test_df = spark.createDataFrame(mereged_test_df1)

In [ ]:
mereged_train_df = mereged_train_df.withColumn("Store", mereged_train_df["Store"].cast(IntegerType()))
mereged_train_df = mereged_train_df.withColumn("Sales", mereged_train_df["Sales"].cast(IntegerType()))
mereged_train_df = mereged_train_df.withColumn("Open", mereged_train_df["Open"].cast(IntegerType()))
mereged_train_df = mereged_train_df.withColumn("Promo", mereged_train_df["Promo"].cast(IntegerType()))
mereged_train_df = mereged_train_df.withColumn("SchoolHoliday", mereged_train_df["SchoolHoliday"].cast(IntegerType()))
mereged_train_df = mereged_train_df.withColumn("CompetitionOpenSinceMonth", mereged_train_df["CompetitionOpenSinceMonth"].cast(IntegerType()))
mereged_train_df = mereged_train_df.withColumn("Promo2", mereged_train_df["Promo2"].cast(IntegerType()))
mereged_train_df = mereged_train_df.withColumn("Customers", mereged_train_df["Customers"].cast(IntegerType()))
mereged_train_df = mereged_train_df.withColumn("Date", mereged_train_df["Date"].cast(IntegerType()))
mereged_train_df = mereged_train_df.withColumn("Promo2SinceWeek", mereged_train_df["Promo2SinceWeek"].cast(IntegerType()))
mereged_train_df = mereged_train_df.withColumn("Promo2SinceYear", mereged_train_df["Promo2SinceYear"].cast(IntegerType()))
mereged_train_df = mereged_train_df.withColumn("PromoInterval", mereged_train_df["PromoInterval"].cast(IntegerType()))
mereged_test_df = mereged_test_df.withColumn("Store", mereged_test_df["Store"].cast(IntegerType()))
mereged_test_df = mereged_test_df.withColumn("Id", mereged_test_df["Id"].cast(IntegerType()))
mereged_test_df = mereged_test_df.withColumn("Open", mereged_test_df["Open"].cast(IntegerType()))
mereged_test_df = mereged_test_df.withColumn("Promo", mereged_test_df["Promo"].cast(IntegerType()))
mereged_test_df = mereged_test_df.withColumn("SchoolHoliday", mereged_test_df["SchoolHoliday"].cast(IntegerType()))
mereged_test_df = mereged_test_df.withColumn("CompetitionOpenSinceMonth", mereged_test_df["CompetitionOpenSinceMonth"].cast(IntegerType()))
mereged_test_df = mereged_test_df.withColumn("Promo2", mereged_test_df["Promo2"].cast(IntegerType()))
mereged_test_df = mereged_test_df.withColumn("Date", mereged_test_df["Date"].cast(IntegerType()))
mereged_test_df = mereged_test_df.withColumn("Promo2SinceWeek", mereged_test_df["Promo2SinceWeek"].cast(IntegerType()))
mereged_test_df = mereged_test_df.withColumn("Promo2SinceYear", mereged_test_df["Promo2SinceYear"].cast(IntegerType()))
mereged_test_df = mereged_test_df.withColumn("PromoInterval", mereged_test_df["PromoInterval"].cast(IntegerType()))

In [ ]:
mereged_train_df2 = mereged_train_df
mereged_train_df2 = mereged_train_df2.filter(mereged_train_df2['Open'] !=0)

featureassember= VectorAssembler(inputCols=['Store', 'DayOfWeek', 'Customers', 'Open', 'Promo', 'StateHoliday', 'SchoolHoliday', 'StoreType', 'Assortment', 'CompetitionDistance', 'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear', 'Promo2', 'Year', 'Month', 'Day'],outputCol="features")
my_data = featureassember.transform(mereged_train_df2)
my_data= my_data.select("features","Sales")
trn,tst = my_data.randomSplit([0.8, 0.2],seed=10)

regressor =LinearRegression(featuresCol="features",labelCol="Sales")
regressor=regressor.fit(my_data)
my_test = regressor.transform(tst)
my_test= my_test.select("features","Sales")

In [ ]:
trainingSummary = regressor.summary
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)
regressor.coefficients

In [ ]:
my_pred1.registerTempTable('my_pred1')
my_pred11 = sqlContext.sql('select Id,prediction from my_pred1')
my_pred11.write.mode("overwrite").format("jdbc")
                                        .option("url",
                                        "jdbc:sqlserver://<your_server>.database.windows.net:1433;database=<your_database>")
                                        .option("dbtable", "dbo.<your_table>")
                                        .option("user", "<user>@<server>")
                                        .option("password", "<password>")
                                        .save()